## Тестовое Computer Vision

В папке cv находятся изображения документов, в которых необходимо распознать все сущности и результат записать в json файл. К примеру, 
для изображения cv/1.jpg результат должен быть таким:

{ 'Выписка из протокола' : 'Комиссии Архитектурного совета по формированию городской среды',
  'дата': '11.07.00', 'номер': '11' , 'пункт': '1',
   'Наименование объекта': 'ТЭО строительства пристройки БНК к зданию средней общеобразовательной школы   № 1650',
   'Авторы проекта': 'Микоян К.С, Макарова Р.Д., Мещерский В.В.', 
   'Генеральная проектная организация': 'МОСКОМАРТ, мастерская № 1',
   'Застройщик': 'ЗАО "ТЭМ-Строй"',
   'Рассмотрение на рабочей комиссии': ' ',
   'Референт': 'Зайцых В.А.',
   'Докладчик': ' ',
   'Выступили': ' ' }

Язык программирования: Python 3.6 или 3.7.

Нефункциональные требования
1. Минимальное необходимое количество библиотек-зависимостей
2. Соответствие исходного кода PEP 8

### Процесс решения задачи

Загружаем необходимые библиотеки.

In [1]:
import os
import re
import cv2
import json
import numpy as np
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'

Создадим необходимые для решения задачи функции.

In [2]:
def save_json(content, json_name):
    
    '''The function saves content according to the dictionary below.'''
    
    content = content[::-1]
    data = {'Выписка из протокола' : content[0],
            'дата': content[3], 'номер': content[2], 'пункт': content[1],
            'Наименование объекта': content[4],
            'Авторы проекта': content[5],
            'Генеральная проектная организация': content[6],
            'Застройщик': content[7],
            'Рассмотрение на рабочей комиссии': '',
            'Референт': content[8],
            'Докладчик': content[9],
            'Выступили': content[10]}
    
    with open(str(json_name) + '.json', 'w', encoding = 'utf-8') as outfile:
        json.dump(data, outfile, ensure_ascii = False)
    outfile.close()

In [3]:
def get_content_and_save_json(path):
    
    '''The function extracts and saves in json-file the content from the image.'''
    
    images_counter = 0
    config = r' -l rus --oem 3 --psm 6'
    
    image_paths = [os.path.join(path, file) for file in os.listdir(path)]
    for image_path in image_paths:
        images_counter += 1
        content = []
        
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

        contoures = cv2.findContours(thresh,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contoures = contoures[0] if len(contoures) == 2 else contoures[1]
        
        for contour in contoures:
            if cv2.contourArea(contour) > 168: 
                epsilon = cv2.arcLength(contour, False)
                approximation = cv2.approxPolyDP(contour, 0.008 * epsilon, False)
                if len(approximation) <= 15:
                    x, y, w, h = cv2.boundingRect(approximation)
                    rectangle = np.array(thresh[y+5: y+h-5, x+5: x+w-5], np.uint8)
                    string = pytesseract.image_to_string(rectangle, config = config)
                    clean_string = re.sub('[^а-яА-Я0-9 "\.\№\,]', ' ', string).strip()
                    content.append(clean_string)
   
        save_json(content, images_counter)

Применяем функцию.

In [4]:
path = 'cv'

In [5]:
get_content_and_save_json(path)